<!--Header-->
<div>
    <div class="row" style="color: #4D4D4D;font-size: 15px;padding-bottom: 20px">
        <div class="col-md-7">
            <img src="http://materials.cv.uoc.edu/cdocent/common/img/logo-uoc.png" alt="Logo UOC" class="img-responsive" style="margin:20px 0px 0px">
        </div>
        <div class="col-md-5">
            <h1 style="margin:15px 0px 0px;font-size: 40px;">Avaluació de models de classificació</h1>
            <div style="text-align:left;margin-top: 5px;"></div>
        </div>
    </div>
    <div class="row" style="background: #FCB517;padding: 10px 20px;">
        <div class="col-md-6">
            <div>PID_00233252</div>
        </div>
        <div class="col-md-6">
            <div style="text-align:right;">Autor: Xavier Duran Albareda <span style="margin-left: 30px;">Coordinació: Julià Minguillón</span></div>
        </div>
    </div>
</div>
<!--/Header-->

## Introducció

Els problemes de classificació són un dels més comuns dins de l'aprenentatge automàtic.

![ML](images/ml.png)

- [model_evaluation](http://scikit-learn.org/stable/modules/model_evaluation.html)
- [hyperparameters-and-model-validation](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html)
- [evaluation](http://magizbox.com/training/machinelearning/site/evaluation/)
- [how-to-predict-telco-churn-with-apache-spark-mllib](https://blog.cloudera.com/blog/2016/02/how-to-predict-telco-churn-with-apache-spark-mllib/)
- [evaluating-a-machine-learning-model](https://www.jeremyjordan.me/evaluating-a-machine-learning-model/)

## Titanic

El conjunt de dades que farem servir correspon al dataset de la competició [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic) de [Kaggle](https://www.kaggle.com/).

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

titanic_clean = pd.read_csv('data/titanic_clean.csv', header = 0)
titanic_clean.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,0,3,1,1,0,0,0,1
1,1,1,0,2,3,1,0,3
2,1,3,0,1,1,0,1,2
3,1,1,0,2,3,0,0,3
4,0,3,1,2,1,0,1,1


La variable _Survived_ és la que volem predir amb el nostre classificador, i només té dues categories: 0 i 1 segons la persona va sobreviure a la catàstrofe o no. Les altres variables corresponen a:

- _Pclass_: la classe de tiquet (1a, 2a o 3a categoria)
- _Sex_: El sexe del passatger (0 correspon a femení; 1 a masculí)
- _Age_: Edat del passatger categoritzada en 5 rangs d'edat:
  - 0 correpon al rang de 0 a 16 anys
  - 1 correpon al rang de 16 a 32 anys
  - 2 correpon al rang de 32 a 48 anys
  - 3 correpon al rang de 48 a 64 anys
  - 4 correpon a més de 64 anys
- _Fare_: Preu del passatge, categoritzat en 5 rangs, de més econòmic a més car
- _Embarked_: Port d'embarcament
- _IsAlone_: Si viatjava sola o no
- _Title_: Títol indicat amb el nom del passatger
  - 0 Master
  - 1 Miss
  - 2 Mr
  - 3 Mrs
  - 4 Rare

In [28]:
cols = ['Pclass']

titanic_categorical = titanic_clean[['Pclass', 'Age', 'Fare', 'Embarked', 'Title']]
titanic_categorical = pd.concat([pd.get_dummies(titanic_categorical[col], prefix=col) for col in titanic_categorical], axis=1)
titanic_categorical.head()

,Pclass_1,Pclass_2,Pclass_3,Age_0,Age_1,Age_2,Age_3,Age_4,Fare_0,Fare_1,Fare_2,Fare_3,Embarked_0,Embarked_1,Embarked_2,Title_1,Title_2,Title_3,Title_4,Title_5
0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0
4,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0


In [37]:
df = pd.concat([titanic_clean[['Survived', 'Sex', 'IsAlone']], titanic_categorical], axis=1)
df.head()

,Survived,Sex,IsAlone,Pclass_1,Pclass_2,Pclass_3,Age_0,Age_1,Age_2,Age_3,...,Fare_2,Fare_3,Embarked_0,Embarked_1,Embarked_2,Title_1,Title_2,Title_3,Title_4,Title_5
0,0,1,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
1,1,0,0,1,0,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0
2,1,0,1,0,0,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,1,0,0,0,0,1,0,...,0,1,1,0,0,0,0,1,0,0
4,0,1,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0


### Conjunts de _train_ i de _test_

Primer de tot, dividirem les nostres dades etiquetades en dos conjunts: el conjunt de _train_, amb el que generarem el nostre model; i el conjunt de _test_, que ens servirà per avaluar el model en unes dades diferents a les que l'han generat.

In [38]:
train, test = train_test_split(df, test_size=0.2)

### Entrenar el model

## Mètriques per avaluar un model de classificació

Al llarg del temps s'han desenvolupat moltes mètriques per avaluar la qualitat d'aquests models de classificació, i fer-ne servir una o una altra dependrà molt de l'apliació concreta que en volguem fer.

### Exactitud

L'exactitud o _accuracy_ és la proporció d'exemples que s'han classificat correctament, o el que és el mateix, en que la categoria que ha predit el nostre model coincideix amb la categoria correcta.

L'avantatge d'aquesta mesura és que és molt senzilla d'entendre, però té moltes limitacions, ja que dóna una visió massa simple del que realment està fent el model. A continuació veurem altres mètriques més complexes i quina motivació tenim per fer-les servir per millorar la nostra visió del que està fent el nostre model en un conjunt de dades determinat.

In [5]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("Accuracy: %.3f (%.3f)") % (results.mean(), results.std())

Accuracy: 0.770 (0.048)


### Matriu de confusió

The confusion matrix is a handy presentation of the accuracy of a model with two or more classes.

The table presents predictions on the x-axis and accuracy outcomes on the y-axis. The cells of the table are the number of predictions made by a machine learning algorithm.

For example, a machine learning algorithm can predict 0 or 1 and each prediction may actually have been a 0 or 1. Predictions for 0 that were actually 0 appear in the cell for prediction=0 and actual=0, whereas predictions for 0 that were actually 1 appear in the cell for prediction = 0 and actual=1. And so on.

You can learn more about the Confusion Matrix on the Wikipedia article.

Below is an example of calculating a confusion matrix for a set of prediction by a model on a test set.

In [3]:
import pandas

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression()
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print(matrix)

[[141  21]
 [ 41  51]]


### Àrea sota la corba ROC

Area under ROC Curve (or AUC for short) is a performance metric for binary classification problems.

The AUC represents a model’s ability to discriminate between positive and negative classes. An area of 1.0 represents a model that made all predictions perfectly. An area of 0.5 represents a model as good as random. Learn more about ROC here.

ROC can be broken down into sensitivity and specificity. A binary classification problem is really a trade-off between sensitivity and specificity.

Sensitivity is the true positive rate also called the recall. It is the number instances from the positive (first) class that actually predicted correctly.
Specificity is also called the true negative rate. Is the number of instances from the negative class (second) class that were actually predicted correctly.
You can learn more about ROC on the Wikipedia page.

The example below provides a demonstration of calculating AUC.

In [7]:
# Cross Validation Classification ROC AUC
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = LogisticRegression()
scoring = 'roc_auc'
results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("AUC: %.3f (%.3f)") % (results.mean(), results.std())

AUC: 0.824 (0.041)


### Informe de classificació

Scikit-learn does provide a convenience report when working on classification problems to give you a quick idea of the accuracy of a model using a number of measures.

The classification_report() function displays the precision, recall, f1-score and support for each class.

The example below demonstrates the report on the binary classification problem.

In [9]:
# Cross Validation Classification Report
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression()
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
report = classification_report(Y_test, predicted)
print(report)

             precision    recall  f1-score   support

        0.0       0.77      0.87      0.82       162
        1.0       0.71      0.55      0.62        92

avg / total       0.75      0.76      0.75       254



## Conclusió

En aquesta activitat hem vist algunes de les visualitzacions més comunes i exemples de com implementar-les en les llibreries `Matplotlib` i `Seaborn`. Una de les qüestions clau per seleccionar el model és escollir una bona mètrica d'avaluació del model que ens indiqui quin serà el rendiment d'aquest en l'àmbit en que l'aplicarem.

<!--Footer-->
 <div style="background: #333333;padding: 35px 0px;margin-top: 25px;">
    <div class="row">
     <div class="col-sm-12">
        <img src="http://materials.cv.uoc.edu/cdocent/common/img/logo-uoc-bottom.png" alt="Logo UOC" class="img-responsive" style="margin: 0 auto; display: block;">
    </div>
</div>
</div>
<!--/Footer-->